<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=200 height=200>
<br>
<font color=0F5298 size=8>
Introduction to Machine Learning <br>
<font color=696880 size=5>
<!-- <br> -->
Computer Engineering Department
<br>
Sharif University of Technology

<font color=696880 size=5>
<br>
CE 40477 - Fall 2024

<font color=GREEN size=5>
<br>
Mahan Bayhaghi & Nikan Vasei
<!-- <br> -->

____

# Ensemble Learning

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Decision Trees

In [33]:
class Node:
    """
    A class representing a node in a decision tree.
    """

    def __init__(self, feature=None, threshold=None, left=None, right=None, gain=None, value=None):
        """
        Initializes a new instance of the Node class.

        Parameters:
        ----------
        feature : int, optional
            The index of the feature used for splitting at this node.
        threshold : float, optional
            The threshold value for splitting the dataset at this node.
        left : Node, optional
            The left child node resulting from the split.
        right : Node, optional
            The right child node resulting from the split.
        gain : float, optional
            The information gain of the split at this node.
        value : int or float, optional
            If this node is a leaf node, this represents the predicted value.
        """
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.gain = gain
        self.value = value

In [34]:
class DecisionTree():
    """
    A decision tree classifier for binary classification problems.
    """

    def __init__(self, min_samples=2, max_depth=2):
        """
        Constructor for DecisionTree class.

        Parameters:
        ----------
        min_samples : int
            Minimum number of samples required to split an internal node.
        max_depth : int
            Maximum depth of the decision tree.
        """
        self.min_samples = min_samples
        self.max_depth = max_depth

    def _split_data(self, dataset, feature, threshold):
        """
        Splits the given dataset into two datasets based on the given feature and threshold.

        Parameters:
        ----------
        dataset : ndarray
            Input dataset.
        feature : int
            Index of the feature to be split on.
        threshold : float
            Threshold value to split the feature on.

        Returns:
        -------
        left_dataset : ndarray
            Subset of the dataset with values less than or equal to the threshold.
        right_dataset : ndarray
            Subset of the dataset with values greater than the threshold.
        """
        # Create empty arrays to store the left and right datasets
        left_dataset = []
        right_dataset = []
        
        # Loop over each row in the dataset and split based on the given feature and threshold
        for row in dataset:
            if row[feature] <= threshold:
                left_dataset.append(row)
            else:
                right_dataset.append(row)

        # Convert the left and right datasets to numpy arrays and return
        left_dataset = np.array(left_dataset)
        right_dataset = np.array(right_dataset)
        return left_dataset, right_dataset

    def _entropy(self, y):
        """
        Computes the entropy of the given label values.

        Parameters:
        ----------
        y : ndarray
            Input label values.

        Returns:
        -------
        entropy : float
            Entropy of the given label values.
        """
        entropy = 0

        # Find the unique label values in y and loop over each value
        labels = np.unique(y)
        for label in labels:
            # Find the examples in y that have the current label
            label_examples = y[y == label]
            # Calculate the ratio of the current label in y
            pl = len(label_examples) / len(y)
            # Calculate the entropy using the current label and ratio
            entropy += -pl * np.log2(pl)

        # Return the final entropy value
        return entropy

    def _information_gain(self, parent, left, right):
        """
        Computes the information gain from splitting the parent dataset into two datasets.

        Parameters:
        ----------
        parent : ndarray
            Input parent dataset.
        left : ndarray
            Subset of the parent dataset after split on a feature.
        right : ndarray
            Subset of the parent dataset after split on a feature.

        Returns:
        -------
        information_gain : float
            Information gain of the split.
        """
        # Set initial information gain to 0
        information_gain = 0
        # Compute entropy for parent
        parent_entropy = self._entropy(parent)
        # Calculate weight for left and right nodes
        weight_left = len(left) / len(parent)
        weight_right= len(right) / len(parent)
        # Compute entropy for left and right nodes
        entropy_left, entropy_right = self._entropy(left), self._entropy(right)
        # Calculate weighted entropy 
        weighted_entropy = weight_left * entropy_left + weight_right * entropy_right
        # Calculate information gain 
        information_gain = parent_entropy - weighted_entropy
        return information_gain
    
    def _best_split(self, dataset, num_features):
        """
        Finds the best split for the given dataset.

        Parameters:
        ----------
        dataset : ndarray
            The dataset to split.
        num_samples : int
            The number of samples in the dataset.
        num_features : int
            The number of features in the dataset.

        Returns:
        -------
        best_split : dict
            A dictionary with the best split feature index, threshold, gain, left, and right datasets.
        """
        # Dictionary to store the best split values
        best_split = {'gain': -1}
        # Loop over all the features
        for feature_index in range(num_features):
            # Get the feature at the current feature_index
            feature_values = dataset[:, feature_index]
            # Get unique values of that feature
            thresholds = np.unique(feature_values)
            # Loop over all values of the feature
            for threshold in thresholds:
                # Get left and right datasets
                left_dataset, right_dataset = self._split_data(dataset, feature_index, threshold)
                # Check if either datasets is empty
                if len(left_dataset) and len(right_dataset):
                    # Get y values of the parent and left, right nodes
                    y, left_y, right_y = dataset[:, -1], left_dataset[:, -1], right_dataset[:, -1]
                    # Compute information gain based on the y values
                    information_gain = self._information_gain(y, left_y, right_y)
                    # Update the best split if conditions are met
                    if information_gain > best_split["gain"]:
                        best_split["feature"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_dataset"] = left_dataset
                        best_split["right_dataset"] = right_dataset
                        best_split["gain"] = information_gain
        return best_split
    
    def _calculate_leaf_value(self, y):
        """
        Calculates the most occurring value in the given list of y values.

        Parameters:
        ----------
        y : list
            The list of y values.

        Returns:
        -------
        most_occurring_value : int or float
            The most occurring value in the list.
        """
        y = list(y)
        # Get the highest present class in the array
        most_occurring_value = max(y, key=y.count)
        return most_occurring_value
    
    def _build_tree(self, dataset, current_depth=0):
        """
        Recursively builds a decision tree from the given dataset.

        Parameters:
        ----------
        dataset : ndarray
            The dataset to build the tree from.
        current_depth : int
            The current depth of the tree.

        Returns:
        -------
        Node : Node
            The root node of the built decision tree.
        """
        # Split the dataset into X, y values
        X, y = dataset[:, :-1], dataset[:, -1]
        n_samples, n_features = X.shape
        # Keeps splitting until stopping conditions are met
        if n_samples >= self.min_samples and current_depth <= self.max_depth:
            # Get the best split
            best_split = self._best_split(dataset, n_features)
            # Check if the gain isn't zero
            if best_split["gain"]:
                # Continue splitting the left and the right child. Increment current depth
                left_node = self._build_tree(best_split["left_dataset"], current_depth + 1)
                right_node = self._build_tree(best_split["right_dataset"], current_depth + 1)
                # Return decision node
                return Node(best_split["feature"], best_split["threshold"], left_node, right_node, best_split["gain"])

        # Compute the leaf node value
        leaf_value = self._calculate_leaf_value(y)
        # Return the leaf node value
        return Node(value=leaf_value)
    
    def fit(self, X, y):
        """
        Builds and fits the decision tree to the given X and y values.

        Parameters:
        ----------
        X : ndarray
            The feature matrix.
        y : ndarray
            The target values.
        """
        dataset = np.concatenate((X, y), axis=1)  
        self.root = self._build_tree(dataset)

    def predict(self, X):
        """
        Predicts the class labels for each instance in the feature matrix X.

        Parameters:
        ----------
        X : ndarray
            The feature matrix to make predictions for.

        Returns:
        -------
        predictions : list
            A list of predicted class labels.
        """
        # Create an empty list to store the predictions
        predictions = []
        # For each instance in X, make a prediction by traversing the tree
        for x in X:
            prediction = self._make_prediction(x, self.root)
            # Append the prediction to the list of predictions
            predictions.append(prediction)
        # Convert the list to a numpy array and return it
        np.array(predictions)
        return predictions
    
    def _make_prediction(self, x, node):
        """
        Traverses the decision tree to predict the target value for the given feature vector.

        Parameters:
        ----------
        x : ndarray
            The feature vector to predict the target value for.
        node : Node
            The current node being evaluated.

        Returns:
        -------
        prediction : int or float
            The predicted target value for the given feature vector.
        """
        # If the node has value i.e it's a leaf node extract it's value
        if node.value != None: 
            return node.value
        else:
            # If it's node a leaf node we'll get it's feature and traverse through the tree accordingly
            feature = x[node.feature]
            if feature <= node.threshold:
                return self._make_prediction(x, node.left)
            else:
                return self._make_prediction(x, node.right)

In this section, we'll use the `Breast Cancer Wisconsin` dataset which is a binary classification dataset, and has 30 different continuous features.

In [35]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True, as_frame=True)

X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


We'll also standardize the data using the `StandardScaler` class.

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = y.values.reshape(-1,1)

Now we can split the dataset into train and test sets.

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape of X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Shape of X_test: {X_train.shape}, y_test: {y_test.shape}")

Shape of X_train: (455, 30), y_train: (455, 1)
Shape of X_test: (455, 30), y_test: (114, 1)


At last we can train a `DecisionTree()` model and evaluate its performance.

In [38]:
from sklearn.metrics import accuracy_score, f1_score

# Create a DT model.
dt_model = DecisionTree(2, 2)

# Fit the DT model to the training data.
dt_model.fit(X_train, y_train)

# Use the trained DT model to make predictions on the test data.
predictions = dt_model.predict(X_test)

# Calculate evaluating metrics for the DT model.
print(f"DT Model's Accuracy: {accuracy_score(y_test, predictions)}")
print(f"DT Model's F1-Score: {f1_score(y_test, predictions)}")

DT Model's Accuracy: 0.9473684210526315
DT Model's F1-Score: 0.9583333333333334


## Random Forest

In [39]:
class RandomForest:
    """
    A random forest classifier.

    Parameters
    ----------
    n_trees : int, default=10
        The number of trees in the random forest.
    max_depth : int, default=2
        The maximum depth of each decision tree in the random forest.
    min_samples : int, default=7
        The minimum number of samples required to split an internal node
        of each decision tree in the random forest.

    Attributes
    ----------
    n_trees : int
        The number of trees in the random forest.
    max_depth : int
        The maximum depth of each decision tree in the random forest.
    min_samples : int
        The minimum number of samples required to split an internal node
        of each decision tree in the random forest.
    trees : list of DecisionTree
        The decision trees in the random forest.
    """

    def __init__(self, n_trees=10, min_samples=7, max_depth=2):
        """
        Initialize the random forest classifier.

        Parameters
        ----------
        n_trees : int, default=10
            The number of trees in the random forest.
        max_depth : int, default=2
            The maximum depth of each decision tree in the random forest.
        min_samples : int, default=7
            The minimum number of samples required to split an internal node
            of each decision tree in the random forest.
        """
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.trees = []

    def fit(self, X, y):
        """
        Build a random forest classifier from the training set (X, y).

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The training input samples.
        y : array-like of shape (n_samples,)
            The target values.

        Returns
        -------
        self : object
            Returns self.
        """
        # Create an empty list to store the trees.
        self.trees = []
        # Concatenate X and y into a single dataset.
        dataset = np.concatenate((X, y), axis=1)
        # Loop over the number of trees.
        for _ in range(self.n_trees):
            # Create a decision tree instance.
            tree = DecisionTree(min_samples=self.min_samples, max_depth=self.max_depth)
            # Sample from the dataset with replacement (bootstrapping).
            dataset_sample = self._bootstrap_samples(dataset)
            # Get the X and y samples from the dataset sample.
            X_sample, y_sample = dataset_sample[:, :-1], dataset_sample[:, -1].reshape(-1, 1)
            # Fit the tree to the X and y samples.
            tree.fit(X_sample, y_sample)
            # Store the tree in the list of trees.
            self.trees.append(tree)

    def _bootstrap_samples(self, dataset):
        """
        Bootstrap the dataset by sampling from it with replacement.

        Parameters
        ----------
        dataset : array-like of shape (n_samples, n_features + 1)
            The dataset to bootstrap.

        Returns
        -------
        dataset_sample : array-like of shape (n_samples, n_features + 1)
            The bootstrapped dataset sample.
        """
        # Get the number of samples in the dataset.
        n_samples = dataset.shape[0]
        # Generate random indices to index into the dataset with replacement.
        np.random.seed(1)
        indices = np.random.choice(n_samples, n_samples, replace=True)
        # Return the bootstrapped dataset sample using the generated indices.
        dataset_sample = dataset[indices]
        return dataset_sample

    def _most_common_label(self, y):
        """
        Return the most common label in an array of labels.

        Parameters
        ----------
        y : array-like of shape (n_samples,)
            The array of labels.

        Returns
        -------
        most_occurring_value : int or float
            The most common label in the array.
        """
        y = list(y)
        # Get the highest present class in the array
        most_occurring_value = max(y, key=y.count)
        return most_occurring_value

    def predict(self, X):
        """
        Predict class for X.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input samples.

        Returns
        -------
        majority_predictions : array-like of shape (n_samples,)
            The predicted classes.
        """
        # Get prediction from each tree in the tree list on the test data
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # Get prediction for the same sample from all trees for each sample in the test data
        preds = np.swapaxes(predictions, 0, 1)
        # Get the most voted value by the trees and store it in the final predictions array
        majority_predictions = np.array([self._most_common_label(pred) for pred in preds])
        return majority_predictions

In [40]:
# Create an RF model
rf_model = RandomForest()

# Fit the RF model to the training data
rf_model.fit(X_train, y_train)

# Use the trained RF model to make predictions on the test data
rf_predictions = rf_model.predict(X_test)

# Calculate evaluating metrics for the RF model
print(f"RF Model's Accuracy: {accuracy_score(y_test, rf_predictions)}")
print(f"RF Model's F1-Score: {f1_score(y_test, rf_predictions)}")

RF Model's Accuracy: 0.9298245614035088
RF Model's F1-Score: 0.9428571428571428


## AdaBoost & XGBoost

You've got familiar with the `AdaBoost` algorithm in the class and also through the slides. <br> Another famous algorithm between the ensemble methods is the `XGBoost`. You may have heard of it before, but to learn more about it, please refer to [this](https://en.wikipedia.org/wiki/XGBoost) link. It should be familiar to you, as you saw decision trees in the class.

You can see the overview of how XGBoost works in the image below:

<dev style="text-align: center">
<img src="./pics/XGBoost.png" />
</dev>

In this section we're going to compare the accuracy of the three methods that we have mentioned above, `Random Forest`, `AdaBoost` and `XGBoost`.

In [41]:
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

ab_model = AdaBoostClassifier(n_estimators=10)
xgb_model = XGBClassifier()

ab_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)

print(f"AB Model's Accuracy: {accuracy_score(y_test, ab_predictions)}")
print(f"AB Model's F1-Score: {f1_score(y_test, ab_predictions)}")

print(f"XGB Model's Accuracy: {accuracy_score(y_test, xgb_predictions)}")
print(f"XGB Model's F1-Score: {f1_score(y_test, xgb_predictions)}")

AB Model's Accuracy: 0.9824561403508771
AB Model's F1-Score: 0.9861111111111112
XGB Model's Accuracy: 0.956140350877193
XGB Model's F1-Score: 0.965034965034965


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
